In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2

print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [4]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 10


# Check the drug

In [5]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [6]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-13 08:30:27] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-13 08:32:01] ax.service.ax_client: Generated new trial 160 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.143624, 'SL_2': 0.0, 'SL_3': 0.020792, 'LL_1': 0.227691, 'LL_2': 0.381234, 'LL_3': 0.0215, 'P_1': 0.053184, 'P_2': 0.605753, 'P_3': 0.774948, 'S_1': 0.0, 'S_2': 0.973637, 'S_3': 0.0, 'Water': 1.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return

23 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000
mean,167.500000,354.0,3.91,67.0,0.427750,0.009687,0.0,0.039625,0.016188,0.059500,0.055625,0.023062,0.029812,0.338875,0.006625,0.380937,0.0,0.612437
std,4.760952,0.0,0.00,0.0,0.106006,0.023916,0.0,0.051078,0.026821,0.085379,0.069558,0.025328,0.083524,0.070628,0.017527,0.087475,0.0,0.080325
min,160.000000,354.0,3.91,67.0,0.175000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.184000,0.000000,0.259000,0.0,0.447000
25%,163.750000,354.0,3.91,67.0,0.403500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000750,0.000000,0.304750,0.000000,0.309000,0.0,0.569500
50%,167.500000,354.0,3.91,67.0,0.435500,0.000000,0.0,0.004000,0.000000,0.017000,0.004000,0.013000,0.000000,0.328000,0.000000,0.380500,0.0,0.619500
75%,171.250000,354.0,3.91,67.0,0.473250,0.000000,0.0,0.093500,0.026750,0.087250,0.109250,0.035750,0.000000,0.399500,0.000250,0.430500,0.0,0.683250
max,175.000000,354.0,3.91,67.0,0.655000,0.087000,0.0,0.115000,0.075000,0.289000,0.178000,0.073000,0.289000,0.444000,0.058000,0.553000,0.0,0.727000


In [8]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000
mean,167.500000,0.354,0.391,0.067,0.962341,0.025419,0.0,0.088357,0.046809,0.167531,0.125621,0.054590,0.100360,0.784371,0.009769,0.596653,0.0,0.954814
std,4.760952,0.000,0.000,0.000,0.108208,0.060862,0.0,0.111637,0.080531,0.271570,0.162955,0.058662,0.283524,0.166377,0.025905,0.207409,0.0,0.180744
min,160.000000,0.354,0.391,0.067,0.607083,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.436275,0.000000,0.342416,0.0,0.277024
25%,163.750000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.001857,0.000000,0.690409,0.000000,0.446379,0.0,1.000000
50%,167.500000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.013727,0.000000,0.036584,0.012329,0.033145,0.000000,0.792511,0.000000,0.587622,0.0,1.000000
75%,171.250000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.221168,0.064617,0.235115,0.241248,0.084195,0.000000,0.922963,0.000193,0.669145,0.0,1.000000
max,175.000000,0.354,0.391,0.067,1.000000,0.204569,0.0,0.238904,0.227691,1.000000,0.423036,0.165831,1.000000,1.000000,0.084420,0.973637,0.0,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.00000
mean,167.500000,354.0,3.91,67.0,149.712500,3.390625,0.0,13.868750,5.665625,20.825000,19.468750,8.071875,10.434375,118.606250,6.625000,380.937500,0.0,612.437500,-0.04375
std,4.760952,0.0,0.00,0.0,37.102091,8.370508,0.0,17.877144,9.387327,29.882704,24.345471,8.864949,29.233478,24.719971,17.526646,87.475306,0.0,80.325147,0.17500
min,160.000000,354.0,3.91,67.0,61.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.400000,0.000000,259.000000,0.0,447.000000,-0.35000
25%,163.750000,354.0,3.91,67.0,141.225000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.262500,0.000000,106.662500,0.000000,309.000000,0.0,569.500000,0.00000
50%,167.500000,354.0,3.91,67.0,152.425000,0.000000,0.0,1.400000,0.000000,5.950000,1.400000,4.550000,0.000000,114.800000,0.000000,380.500000,0.0,619.500000,0.00000
75%,171.250000,354.0,3.91,67.0,165.637500,0.000000,0.0,32.725000,9.362500,30.537500,38.237500,12.512500,0.000000,139.825000,0.250000,430.500000,0.0,683.250000,0.00000
max,175.000000,354.0,3.91,67.0,229.250000,30.450000,0.0,40.250000,26.250000,101.150000,62.300000,25.550000,101.150000,155.400000,58.000000,553.000000,0.0,727.000000,0.35000


In [ ]:
# dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       2.4
SL_1       0.1
SL_2       0.0
SL_3       0.2
LL_1       0.1
LL_2       0.3
LL_3       0.3
P_1        0.1
P_2        0.2
P_3        1.9
S_1        0.1
S_2        6.1
S_3        0.0
Water      9.8
Solvent   -0.0
dtype: float64

In [11]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [12]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-13 08:55:41] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_10.json`.


# Generate new protocol

In [13]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [14]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [15]:
updated_script

'protocol/iteration_10_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,160,354,3.91,67,0.310,0.044,0,0.006,0.071,0.118,0.007,0.016,0.188,0.240,0.000,0.493,0,0.507,9,0
1,161,354,3.91,67,0.456,0.000,0,0.104,0.035,0.034,0.000,0.047,0.000,0.323,0.001,0.309,0,0.690,7,0
2,162,354,3.91,67,0.464,0.000,0,0.109,0.004,0.023,0.000,0.001,0.000,0.399,0.044,0.294,0,0.662,7,0
3,163,354,3.91,67,0.448,0.000,0,0.000,0.000,0.000,0.125,0.011,0.000,0.416,0.000,0.396,0,0.604,4,0
4,164,354,3.91,67,0.413,0.000,0,0.000,0.000,0.000,0.175,0.032,0.000,0.380,0.000,0.391,0,0.609,4,0
5,165,354,3.91,67,0.444,0.000,0,0.000,0.000,0.011,0.101,0.000,0.000,0.444,0.000,0.379,0,0.621,4,0
6,166,354,3.91,67,0.375,0.000,0,0.090,0.075,0.077,0.000,0.059,0.000,0.324,0.000,0.316,0,0.684,6,0
7,167,354,3.91,67,0.520,0.000,0,0.115,0.000,0.070,0.000,0.015,0.000,0.280,0.000,0.309,0,0.691,5,0
8,168,354,3.91,67,0.655,0.000,0,0.000,0.000,0.000,0.059,0.000,0.000,0.286,0.000,0.361,0,0.639,3,0
9,169,354,3.91,67,0.501,0.000,0,0.000,0.000,0.000,0.104,0.031,0.000,0.365,0.000,0.382,0,0.618,4,0


In [17]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,163.541352,0.230545
1,Good,A2,167.142701,0.191680
2,Good,A3,159.060029,0.208521
3,Caution,A4,180.503596,0.570796
4,Caution,A5,197.976678,0.570796
5,Caution,A6,177.722745,0.570796
6,Caution,B1,168.996280,0.570796
7,Caution,B2,230.797347,0.570796
8,Caution,B3,171.501309,0.570796
9,Good,B4,121.338893,0.272562


In [18]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,163.248027,4.049312,0.210249,0.019490,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,122.136976,6.105028,0.285187,0.069810,1
4,Formulation 5,150.378708,8.046551,0.217494,0.016806,1
5,Formulation 6,93.156349,3.915927,0.330675,0.035378,1
6,Formulation 7,187.501366,4.078783,0.365562,0.017082,1
7,Formulation 8,223.494084,6.015187,0.317723,0.010343,1
8,Formulation 9,54.968439,1.420117,0.536572,0.009093,1
9,Formulation 10,97.061065,6.554498,0.374576,0.041265,1


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,160,354,3.91,67,0.310,0.044,0,0.006,0.071,0.118,...,0,0.507,9,0,Formulation 1,163.248027,4.049312,0.210249,0.019490,1
1,161,354,3.91,67,0.456,0.000,0,0.104,0.035,0.034,...,0,0.690,7,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,162,354,3.91,67,0.464,0.000,0,0.109,0.004,0.023,...,0,0.662,7,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,163,354,3.91,67,0.448,0.000,0,0.000,0.000,0.000,...,0,0.604,4,0,Formulation 4,122.136976,6.105028,0.285187,0.069810,1
4,164,354,3.91,67,0.413,0.000,0,0.000,0.000,0.000,...,0,0.609,4,0,Formulation 5,150.378708,8.046551,0.217494,0.016806,1
5,165,354,3.91,67,0.444,0.000,0,0.000,0.000,0.011,...,0,0.621,4,0,Formulation 6,93.156349,3.915927,0.330675,0.035378,1
6,166,354,3.91,67,0.375,0.000,0,0.090,0.075,0.077,...,0,0.684,6,0,Formulation 7,187.501366,4.078783,0.365562,0.017082,1
7,167,354,3.91,67,0.520,0.000,0,0.115,0.000,0.070,...,0,0.691,5,0,Formulation 8,223.494084,6.015187,0.317723,0.010343,1
8,168,354,3.91,67,0.655,0.000,0,0.000,0.000,0.000,...,0,0.639,3,0,Formulation 9,54.968439,1.420117,0.536572,0.009093,1
9,169,354,3.91,67,0.501,0.000,0,0.000,0.000,0.000,...,0,0.618,4,0,Formulation 10,97.061065,6.554498,0.374576,0.041265,1


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,160,354,3.91,67,0.310,0.044,0,0.006,0.071,0.118,...,9,0,Formulation 1,163.248027,4.049312,0.210249,0.019490,1,620.0,0
1,161,354,3.91,67,0.456,0.000,0,0.104,0.035,0.034,...,7,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,162,354,3.91,67,0.464,0.000,0,0.109,0.004,0.023,...,7,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,163,354,3.91,67,0.448,0.000,0,0.000,0.000,0.000,...,4,0,Formulation 4,122.136976,6.105028,0.285187,0.069810,1,896.0,0
4,164,354,3.91,67,0.413,0.000,0,0.000,0.000,0.000,...,4,0,Formulation 5,150.378708,8.046551,0.217494,0.016806,1,826.0,0
5,165,354,3.91,67,0.444,0.000,0,0.000,0.000,0.011,...,4,0,Formulation 6,93.156349,3.915927,0.330675,0.035378,1,888.0,0
6,166,354,3.91,67,0.375,0.000,0,0.090,0.075,0.077,...,6,0,Formulation 7,187.501366,4.078783,0.365562,0.017082,1,750.0,0
7,167,354,3.91,67,0.520,0.000,0,0.115,0.000,0.070,...,5,0,Formulation 8,223.494084,6.015187,0.317723,0.010343,1,1040.0,0
8,168,354,3.91,67,0.655,0.000,0,0.000,0.000,0.000,...,3,0,Formulation 9,54.968439,1.420117,0.536572,0.009093,1,1310.0,0
9,169,354,3.91,67,0.501,0.000,0,0.000,0.000,0.000,...,4,0,Formulation 10,97.061065,6.554498,0.374576,0.041265,1,1002.0,0


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.163248,0.004049,0.310,0.0,0.210249,0.019490,0.750000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
3,1,0.122137,0.006105,0.448,0.0,0.285187,0.069810,0.333333
4,1,0.150379,0.008047,0.413,0.0,0.217494,0.016806,0.333333
5,1,0.093156,0.003916,0.444,0.0,0.330675,0.035378,0.333333
6,1,0.187501,0.004079,0.375,0.0,0.365562,0.017082,0.500000
7,1,0.223494,0.006015,0.520,0.0,0.317723,0.010343,0.416667
8,1,0.054968,0.001420,0.655,0.0,0.536572,0.009093,0.250000
9,1,0.097061,0.006554,0.501,0.0,0.374576,0.041265,0.333333


In [22]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.163248,0.004049,0.310,0.0,0.210249,0.019490,0.750000
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
3,1,0.122137,0.006105,0.448,0.0,0.285187,0.069810,0.333333
4,1,0.150379,0.008047,0.413,0.0,0.217494,0.016806,0.333333
5,1,0.093156,0.003916,0.444,0.0,0.330675,0.035378,0.333333
6,1,0.187501,0.004079,0.375,0.0,0.365562,0.017082,0.500000
7,1,0.223494,0.006015,0.520,0.0,0.317723,0.010343,0.416667
8,1,0.054968,0.001420,0.655,0.0,0.536572,0.009093,0.250000
9,1,0.097061,0.006554,0.501,0.0,0.374576,0.041265,0.333333


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,160,354,3.91,67,0.310,0.044,0,0.006,0.071,0.118,...,0.750000,0,Formulation 1,0.163248,0.004049,0.210249,0.019490,1,0.310,0
1,161,354,3.91,67,0.456,0.000,0,0.104,0.035,0.034,...,0.583333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,162,354,3.91,67,0.464,0.000,0,0.109,0.004,0.023,...,0.583333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,163,354,3.91,67,0.448,0.000,0,0.000,0.000,0.000,...,0.333333,0,Formulation 4,0.122137,0.006105,0.285187,0.069810,1,0.448,0
4,164,354,3.91,67,0.413,0.000,0,0.000,0.000,0.000,...,0.333333,0,Formulation 5,0.150379,0.008047,0.217494,0.016806,1,0.413,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-13 11:01:47] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 160 with data: {'Solu': (0.31, 0), 'Size': (0.163248, 0.004049), 'PDI': (0.210249, 0.01949), 'Complexity': (0.75, 0)}.
[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 161 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.583333, 0)}.
[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 162 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.583333, 0)}.
[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 163 with data: {'Solu': (0.448, 0), 'Size': (0.122137, 0.006105), 'PDI': (0.285187, 0.06981), 'Complexity': (0.333333, 0)}.
[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 164 with data: {'Solu': (0.413, 0), 'Size': (0.150379, 0.008047), 'PDI': (0.217494, 0.016806), 'Complexity': (0.333333, 0)}.
[INFO 02-13 11:01:49] ax.service.ax_client: Completed trial 165 with data: {'Solu': (0.444, 0), 'Size': (0.093156, 0.003916), 